# SLAM Unit F
Finally, we will look at *SLAM!* Up to now, we assumed the map being given. In this unit, we will build the map and localize ourselves simultaneously.

## From localization to SLAM

In [90]:
# If you don't see a video below, run this cell.
# YouTube = True  # Uncomment to get YouTube videos instead of TIB AV.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/NWxVKxZQBFs" if "YouTube" in globals() else "//av.tib.eu/player/49024",
       width=560, height=315)

In [91]:
# Will the landmark position error approach zero if the number
# of observations goes to infinity? Answer "Yes" or "No".
error_goes_to_zero = "No"
# YOUR CODE HERE
#raise NotImplementedError()

In [92]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) == reference
assert public_string_test("well,"+error_goes_to_zero.strip().lower(), '75c69246'), "Oh no, your answer is wrong!"

In [93]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/lzjP9t_UPV8" if "YouTube" in globals() else "//av.tib.eu/player/49025",
       width=560, height=315)

In [94]:
# For the online SLAM, how many variables are in our system state?
online_variables = "103"
# YOUR CODE HERE
#raise NotImplementedError()

In [95]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) == reference
assert public_string_test(online_variables.strip().lower(), '659067d9'), "Oh no, your answer is wrong!"

In [96]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/wniJgJG0vNE" if "YouTube" in globals() else "//av.tib.eu/player/49026",
       width=560, height=315)

In [97]:
# For the full SLAM, how many variables are in our system state?
fullstate_variables = "700"
# YOUR CODE HERE
#raise NotImplementedError()

In [98]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) == reference
assert public_string_test(fullstate_variables.strip().lower(), 'a78cbae5'), "Oh no, your answer is wrong!"

## The EKF SLAM prediction step

In [99]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/o9b9Lz80xvw" if "YouTube" in globals() else "//av.tib.eu/player/49027",
       width=560, height=315)

### Programming assignment: program the prediction step (15 Points).

In [100]:
# EKF SLAM - prediction step.
# slam_09_a_slam_prediction
from lego_robot import *
from math import sin, cos, pi, atan2, sqrt
from numpy import *

class ExtendedKalmanFilterSLAM_1:
    def __init__(self, state, covariance,
                 robot_width, scanner_displacement,
                 control_motion_factor, control_turn_factor):
        # The state. This is the core data of the Kalman filter.
        self.state = state
        self.covariance = covariance

        # Some constants.
        self.robot_width = robot_width
        self.scanner_displacement = scanner_displacement
        self.control_motion_factor = control_motion_factor
        self.control_turn_factor = control_turn_factor

        # Currently, the number of landmarks is zero.
        self.number_of_landmarks = 0

    @staticmethod
    def g(state, control, w):
        x, y, theta = state
        l, r = control
        if r != l:
            alpha = (r - l) / w
            rad = l/alpha
            g1 = x + (rad + w/2.)*(sin(theta+alpha) - sin(theta))
            g2 = y + (rad + w/2.)*(-cos(theta+alpha) + cos(theta))
            g3 = (theta + alpha + pi) % (2*pi) - pi
        else:
            g1 = x + l * cos(theta)
            g2 = y + l * sin(theta)
            g3 = theta

        return array([g1, g2, g3])

    @staticmethod
    def dg_dstate(state, control, w):
        theta = state[2]
        l, r = control
        if r != l:
            alpha = (r-l)/w
            theta_ = theta + alpha
            rpw2 = l/alpha + w/2.0
            m = array([[1.0, 0.0, rpw2*(cos(theta_) - cos(theta))],
                       [0.0, 1.0, rpw2*(sin(theta_) - sin(theta))],
                       [0.0, 0.0, 1.0]])
        else:
            m = array([[1.0, 0.0, -l*sin(theta)],
                       [0.0, 1.0,  l*cos(theta)],
                       [0.0, 0.0,  1.0]])
        return m

    @staticmethod
    def dg_dcontrol(state, control, w):
        theta = state[2]
        l, r = tuple(control)
        if r != l:
            rml = r - l
            rml2 = rml * rml
            theta_ = theta + rml/w
            dg1dl = w*r/rml2*(sin(theta_)-sin(theta))  - (r+l)/(2*rml)*cos(theta_)
            dg2dl = w*r/rml2*(-cos(theta_)+cos(theta)) - (r+l)/(2*rml)*sin(theta_)
            dg1dr = (-w*l)/rml2*(sin(theta_)-sin(theta)) + (r+l)/(2*rml)*cos(theta_)
            dg2dr = (-w*l)/rml2*(-cos(theta_)+cos(theta)) + (r+l)/(2*rml)*sin(theta_)
            
        else:
            dg1dl = 0.5*(cos(theta) + l/w*sin(theta))
            dg2dl = 0.5*(sin(theta) - l/w*cos(theta))
            dg1dr = 0.5*(-l/w*sin(theta) + cos(theta))
            dg2dr = 0.5*(l/w*cos(theta) + sin(theta))

        dg3dl = -1.0/w
        dg3dr = 1.0/w
        m = array([[dg1dl, dg1dr], [dg2dl, dg2dr], [dg3dl, dg3dr]])
            
        return m

    def predict(self, control):
        """The prediction step of the Kalman filter."""
        # covariance' = G * covariance * GT + R
        # where R = V * (covariance in control space) * VT.
        # Covariance in control space depends on move distance.
        G3 = self.dg_dstate(self.state, control, self.robot_width)
        left, right = control
        left_var = (self.control_motion_factor * left)**2 +\
                   (self.control_turn_factor * (left-right))**2
        right_var = (self.control_motion_factor * right)**2 +\
                    (self.control_turn_factor * (left-right))**2
        control_covariance = diag([left_var, right_var])
        V = self.dg_dcontrol(self.state, control, self.robot_width)
        R3 = dot(V, dot(control_covariance, V.T))

        # Hints:
        # - The number of landmarks is self.number_of_landmarks.
        # - eye(n) is the numpy function which returns a n x n identity matrix.
        # - zeros((n,n)) returns a n x n matrix which is all zero.
        # - If M is a matrix, M[0:2,1:5] returns the submatrix which consists
        #   of the rows 0 and 1 (but not 2) and the columns 1, 2, 3, 4.
        #   This submatrix operator can be used on either side of an assignment.
        # - Similarly for vectors: v[1:3] returns the vector consisting of the
        #   elements 1 and 2, but not 3.
        # - All matrix and vector indices start at 0.

        # Now enlarge G3 (getting G) and R3 (getting R) to accomodate all
        # landmarks. Then, compute the new covariance matrix and the new state
        # and assign them to the class using something like:
        #    self.covariance = ...
        #    self.state = ...

        # YOUR CODE HERE
        #raise NotImplementedError()
        
        n = 3 + 2 * self.number_of_landmarks
        G = eye(n)
        G[0:3,0:3] = G3
        
        R = zeros((n, n))
        R[0:3,0:3] = R3
        
        self.covariance = dot(G, dot(self.covariance, G.T)) + R  
        self.state[0:3] = self.g(self.state[0:3], control, self.robot_width)  

    @staticmethod
    def get_error_ellipse(covariance):
        """Return the position covariance (which is the upper 2x2 submatrix)
           as a triple: (main_axis_angle, stddev_1, stddev_2), where
           main_axis_angle is the angle (pointing direction) of the main axis,
           along which the standard deviation is stddev_1, and stddev_2 is the
           standard deviation along the other (orthogonal) axis."""
        eigenvals, eigenvects = linalg.eig(covariance[0:2,0:2])
        angle = atan2(eigenvects[1,0], eigenvects[0,0])
        return (angle, sqrt(eigenvals[0]), sqrt(eigenvals[1]))        


if __name__ == '__main__':
    # Robot constants.
    scanner_displacement = 30.0
    ticks_to_mm = 0.349
    robot_width = 155.0

    # Filter constants.
    control_motion_factor = 0.35  # Error in motor control.
    control_turn_factor = 0.6  # Additional error due to slip when turning.

    # Arbitrary start position.
    initial_state = array([500.0, 0.0, 45.0 / 180.0 * pi])

    # Covariance at start position.
    initial_covariance = zeros((3,3))

    # Setup filter.
    kf = ExtendedKalmanFilterSLAM_1(initial_state, initial_covariance,
                                    robot_width, scanner_displacement,
                                    control_motion_factor, control_turn_factor)

    # Read data.
    logfile = LegoLogfile()
    logfile.read("robot4_motors.txt")

    # Loop over all motor tick records and all measurements and generate
    # filtered positions and covariances.
    # This is the EKF SLAM loop.
    f = open("ekf_slam_prediction.txt", "w")
    for i in range(len(logfile.motor_ticks)):
        # Prediction.
        control = array(logfile.motor_ticks[i]) * ticks_to_mm
        kf.predict(control)

        # End of EKF SLAM - from here on, data is written.

        # Output the center of the scanner, not the center of the robot.
        print("F %f %f %f" % \
            tuple(kf.state[0:3] + [scanner_displacement * cos(kf.state[2]),
                                   scanner_displacement * sin(kf.state[2]),
                                   0.0]), file=f)
        # Write covariance matrix in angle stddev1 stddev2 stddev-heading form
        e = kf.get_error_ellipse(kf.covariance)
        print("E %f %f %f %f" % (e + (sqrt(kf.covariance[2,2]),)), file=f)

    f.close()

### First, the test.

In [101]:
from hashlib import shake_128
import inspect
from numpy import *
from numpy.random import default_rng
from lego_robot import *
from slam_f_library import *

def public_test(the_ekf_slam_class):
    m = shake_128()
    m.update(inspect.getsource(EKF_SLAM_base).encode())
    if m.hexdigest(8) != "05aaff349ba02a8c":
        raise ImportError("slam_f_library")

    # Setup filter.
    rng = default_rng()
    N = rng.integers(3,10)
    robot_state = rng.uniform(0,2000,3)
    robot_state[2] %= pi
    landmark_state = rng.uniform(0,2000,2*N)
    cov = rng.uniform(0,1,(3+2*N, 3+2*N))
    cov += cov.T
    control = rng.uniform(0,10,2)
    kf = the_ekf_slam_class(concatenate((robot_state, landmark_state)),
        cov.copy(), 155.0, 30.0, 0.35, 0.6)
    kf.number_of_landmarks = N
    esb = EKF_SLAM_base(kf)
    kf.predict(control) # Run user function.

    # Check state.
    if not (allclose(kf.state[0:3], esb.g(robot_state, control, esb.robot_width))\
       and allclose(kf.state[3:], landmark_state)):
        print("Predicted state is wrong.")
        return False
    # Check covariance.
    G3, R3 = esb.predict_g3_r3(control)
    if not (allclose(kf.covariance[:3,:3], G3@cov[:3,:3]@G3.T + R3)\
        and allclose(kf.covariance[:3,3:], G3@cov[:3,3:])\
        and allclose(kf.covariance[3:,:3], cov[3:,:3]@G3.T)\
        and allclose(kf.covariance[3:,3:], cov[3:,3:])):
        print("Covariance matrix is wrong.")
        return False
    return True
assert(public_test(ExtendedKalmanFilterSLAM_1))

### Then, let's have a look at the results.
You should see results identical to the ones in the video.

In [102]:
# Execute this to run the interactive viewer.
import ipy_logfile_viewer as lfv
v = lfv.IPYLogfileViewer(files=["ekf_slam_prediction.txt"])

## Adding Landmarks

In [103]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/TQwzG_dy-Ao" if "YouTube" in globals() else "//av.tib.eu/player/49028",
       width=560, height=315)

### Programming assignment: adding one landmark (15 Points).

In [104]:
# EKF SLAM - adding one landmark.
# slam_09_b_slam_add_landmark
from lego_robot import *
from numpy import *
from slam_f_library import write_cylinders, write_error_ellipses

class ExtendedKalmanFilterSLAM_2(ExtendedKalmanFilterSLAM_1):

    def add_landmark_to_state(self, initial_coords):
        """Enlarge the current state and covariance matrix to include one more
           landmark, which is given by its initial_coords (an (x, y) tuple).
           Returns the index of the newly added landmark."""

        # --->>> Add code to augment the robot's state and
        #        covariance matrix.
        #        Initialize the state with the given initial_coords and the
        #        covariance with 1e10 (as an approximation for "infinity").
        # Hints:
        # - If M is a matrix, use M[i:j,k:l] to obtain the submatrix of
        #   rows i to j-1 and colums k to l-1. This can be used on the left and
        #   right side of the assignment operator.
        # - zeros(n) gives a zero vector of length n, eye(n) an n x n identity
        #   matrix.
        # - Do not forget to increment self.number_of_landmarks.
        # - Do not forget to return the index of the newly added landmark. I.e.,
        #   the first call should return 0, the second should return 1.

        #index_of_added_landmark = -1  # Replace this by your code.

        # YOUR CODE HERE
        #raise NotImplementedError()
        
        index_of_added_landmark = self.number_of_landmarks
        self.number_of_landmarks = index_of_added_landmark+1
        
        state_old = self.state
        covariance_old = self.covariance

        # state        
        state_new = append(state_old,initial_coords)
        self.state = state_new

        #covariance
        covariance_new = zeros((3+2*(index_of_added_landmark+1),3+2*(index_of_added_landmark+1)))
        covariance_new[0:3+2*index_of_added_landmark,0:3+2*index_of_added_landmark] = covariance_old
        covariance_new[3+2*index_of_added_landmark:,3+2*index_of_added_landmark:] = diag([1e10,1e10])
        self.covariance = covariance_new
        
        return index_of_added_landmark  


    def get_landmarks(self):
        """Returns a list of (x, y) tuples of all landmark positions."""
        return ([(self.state[3+2*j], self.state[3+2*j+1])
                 for j in range(self.number_of_landmarks)])

    def get_landmark_error_ellipses(self):
        """Returns a list of all error ellipses, one for each landmark."""
        ellipses = []
        for i in range(self.number_of_landmarks):
            j = 3 + 2 * i
            ellipses.append(self.get_error_ellipse(
                self.covariance[j:j+2, j:j+2]))
        return ellipses    

if __name__ == '__main__':
    # Robot constants.
    scanner_displacement = 30.0
    ticks_to_mm = 0.349
    robot_width = 155.0

    # Filter constants.
    control_motion_factor = 0.35  # Error in motor control.
    control_turn_factor = 0.6  # Additional error due to slip when turning.

    # Arbitrary start position.
    initial_state = array([500.0, 0.0, 45.0 / 180.0 * pi])

    # Covariance at start position.
    initial_covariance = zeros((3,3))

    # Setup filter.
    kf = ExtendedKalmanFilterSLAM_2(initial_state, initial_covariance,
                                    robot_width, scanner_displacement,
                                    control_motion_factor, control_turn_factor)

    # Just to test the algorithm, add one landmark.
    kf.add_landmark_to_state((400.0, 700.0))
    # To make the error ellipse visible, set a smaller variance.
    kf.covariance[-2,-2] = 300.0**2  # 300 mm in x.
    kf.covariance[-1,-1] = 500.0**2  # 500 mm in y.

    # Read data.
    logfile = LegoLogfile()
    logfile.read("robot4_motors.txt")

    # Loop over all motor tick records and all measurements and generate
    # filtered positions and covariances.
    # This is the EKF SLAM loop.
    f = open("ekf_slam_add_landmarks.txt", "w")
    for i in range(len(logfile.motor_ticks)):
        # Prediction.
        control = array(logfile.motor_ticks[i]) * ticks_to_mm
        kf.predict(control)

        # End of EKF SLAM - from here on, data is written.

        # Output the center of the scanner, not the center of the robot.
        print("F %f %f %f" % \
            tuple(kf.state[0:3] + [scanner_displacement * cos(kf.state[2]),
                                   scanner_displacement * sin(kf.state[2]),
                                   0.0]), file=f)
        # Write covariance matrix in angle stddev1 stddev2 stddev-heading form
        e = kf.get_error_ellipse(kf.covariance)
        print("E %f %f %f %f" % (e + (sqrt(kf.covariance[2,2]),)), file=f)
        # Write estimates of landmarks.
        write_cylinders(f, "W C", kf.get_landmarks())
        # Write error ellipses of landmarks.
        write_error_ellipses(f, "W E", kf.get_landmark_error_ellipses())

    f.close()

### First, the test.

In [105]:
from numpy import *
from numpy.random import default_rng

def public_test(the_ekf_slam_class):
    # Setup filter.
    rng = default_rng()
    N = rng.integers(3,10)
    M = 3+2*N
    robot_state = rng.uniform(0,2000,3+2*N)
    robot_state[2] %= pi
    cov = rng.uniform(-1,1,(3+2*N, 3+2*N))
    cov += cov.T
    kf = the_ekf_slam_class(
        robot_state.copy(),cov.copy(), 155.0, 30.0, 0.35, 0.6)
    kf.number_of_landmarks = N
    landmark = rng.uniform(0,2000,2)
    li = kf.add_landmark_to_state(landmark.copy())

    # Now check.
    if not (allclose(kf.state[:M], robot_state)\
        and allclose(kf.state[-2:], landmark)):
        print("Wrong state.")
        return False
    if not (allclose(kf.covariance[:M,:M], cov)\
        and allclose(kf.covariance[-2:,-2:], eye(2)*1e+10)\
        and allclose(kf.covariance[:M,-2:], zeros((M,2)))\
        and allclose(kf.covariance[M:,:M], zeros((2,M)))):
        print("Wrong covariance matrix.")
        return False
    if not (li==N and kf.number_of_landmarks==N+1):
        print("Landmark numbers wrong.")
        return False
    return True
assert(public_test(ExtendedKalmanFilterSLAM_2))

### Then let's look at the results (the same as before, plus one landmark).
You should see results identical to the ones in the video.

In [106]:
# Execute this to run the interactive viewer.
import ipy_logfile_viewer as lfv
v = lfv.IPYLogfileViewer(files=["ekf_slam_add_landmarks.txt"])

## Adding the correction step.

In [107]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/dlyzXjc8pRA" if "YouTube" in globals() else "//av.tib.eu/player/49029",
       width=560, height=315)

### Programming assignment: program the correction step (15 Points).

In [108]:
# EKF SLAM - prediction, landmark assignment and correction.
#
# slam_09_c_slam_correction
# Claus Brenner, 20 JAN 13
# 24 SEP 2020 CB  Modified for Python3.
from lego_robot import *
from numpy import *
from slam_f_library import get_observations, write_cylinders,\
     write_error_ellipses

class ExtendedKalmanFilterSLAM_3(ExtendedKalmanFilterSLAM_2):
    def __init__(self, state, covariance,
                 robot_width, scanner_displacement,
                 control_motion_factor, control_turn_factor,
                 measurement_distance_stddev, measurement_angle_stddev):
        # Init from superclass.
        super().__init__(state, covariance,
                 robot_width, scanner_displacement,
                 control_motion_factor, control_turn_factor)
        # Some constants.
        self.measurement_distance_stddev = measurement_distance_stddev
        self.measurement_angle_stddev = measurement_angle_stddev


    @staticmethod
    def h(state, landmark, scanner_displacement):
        """Takes a (x, y, theta) state and a (x, y) landmark, and returns the
           measurement (range, bearing)."""
        dx = landmark[0] - (state[0] + scanner_displacement * cos(state[2]))
        dy = landmark[1] - (state[1] + scanner_displacement * sin(state[2]))
        r = sqrt(dx * dx + dy * dy)
        alpha = (arctan2(dy, dx) - state[2] + pi) % (2*pi) - pi

        return array([r, alpha])

    @staticmethod
    def dh_dstate(state, landmark, scanner_displacement):
        theta = state[2]
        cost, sint = cos(theta), sin(theta)
        dx = landmark[0] - (state[0] + scanner_displacement * cost)
        dy = landmark[1] - (state[1] + scanner_displacement * sint)
        q = dx * dx + dy * dy
        sqrtq = sqrt(q)
        drdx = -dx / sqrtq
        drdy = -dy / sqrtq
        drdtheta = (dx * sint - dy * cost) * scanner_displacement / sqrtq
        dalphadx =  dy / q
        dalphady = -dx / q
        dalphadtheta = -1 - scanner_displacement / q * (dx * cost + dy * sint)

        return array([[drdx, drdy, drdtheta],
                      [dalphadx, dalphady, dalphadtheta]])

    def correct(self, measurement, landmark_index):
        """The correction step of the Kalman filter."""
        # Kalman gain.
        landmark = self.state[3+2*landmark_index : 3+2*landmark_index+2]
        H3 = self.dh_dstate(self.state, landmark, self.scanner_displacement)

        H = H3  # Replace this.

        # YOUR CODE HERE
        
        H = zeros((2,3+2*self.number_of_landmarks))
        H[0:2,0:3] = H3
        H[0:2,3+2*landmark_index:3+2*landmark_index+2] = -1*H3[0:2,0:2]
        #raise NotImplementedError()

        # This is the old code from the EKF - no modification necessary!
        Q = diag([self.measurement_distance_stddev**2,
                  self.measurement_angle_stddev**2])
        K = dot(self.covariance,
                dot(H.T, linalg.inv(dot(H, dot(self.covariance, H.T)) + Q)))
        innovation = array(measurement) -\
                     self.h(self.state, landmark, self.scanner_displacement)
        innovation[1] = (innovation[1] + pi) % (2*pi) - pi
        self.state = self.state + dot(K, innovation)
        self.covariance = dot(eye(size(self.state)) - dot(K, H),
                              self.covariance)     


if __name__ == '__main__':
    # Robot constants.
    scanner_displacement = 30.0
    ticks_to_mm = 0.349
    robot_width = 155.0

    # Cylinder extraction and matching constants.
    minimum_valid_distance = 20.0
    depth_jump = 100.0
    cylinder_offset = 90.0
    max_cylinder_distance = 500.0

    # Filter constants.
    control_motion_factor = 0.35  # Error in motor control.
    control_turn_factor = 0.6  # Additional error due to slip when turning.
    measurement_distance_stddev = 600.0  # Distance measurement error of cylinders.
    measurement_angle_stddev = 45. / 180.0 * pi  # Angle measurement error.

    # Arbitrary start position.
    initial_state = array([500.0, 0.0, 45.0 / 180.0 * pi])

    # Covariance at start position.
    initial_covariance = zeros((3,3))

    # Setup filter.
    kf = ExtendedKalmanFilterSLAM_3(initial_state, initial_covariance,
                                    robot_width, scanner_displacement,
                                    control_motion_factor, control_turn_factor,
                                    measurement_distance_stddev,
                                    measurement_angle_stddev)

    # Read data.
    logfile = LegoLogfile()
    logfile.read("robot4_motors.txt")
    logfile.read("robot4_scan.txt")

    # Loop over all motor tick records and all measurements and generate
    # filtered positions and covariances.
    # This is the EKF SLAM loop.
    f = open("ekf_slam_correction.txt", "w")
    for i in range(len(logfile.motor_ticks)):
        # Prediction.
        control = array(logfile.motor_ticks[i]) * ticks_to_mm
        kf.predict(control)

        # Correction.
        observations = get_observations(
            logfile.scan_data[i],
            depth_jump, minimum_valid_distance, cylinder_offset,
            kf, max_cylinder_distance)
        for obs in observations:
            measurement, cylinder_world, cylinder_scanner, cylinder_index = obs
            if cylinder_index == -1:
                cylinder_index = kf.add_landmark_to_state(cylinder_world)
            kf.correct(measurement, cylinder_index)

        # End of EKF SLAM - from here on, data is written.

        # Output the center of the scanner, not the center of the robot.
        print("F %f %f %f" % \
            tuple(kf.state[0:3] + [scanner_displacement * cos(kf.state[2]),
                                   scanner_displacement * sin(kf.state[2]),
                                   0.0]), file=f)
        # Write covariance matrix in angle stddev1 stddev2 stddev-heading form.
        e = kf.get_error_ellipse(kf.covariance)
        print("E %f %f %f %f" % (e + (sqrt(kf.covariance[2,2]),)), file=f)
        # Write estimates of landmarks.
        write_cylinders(f, "W C", kf.get_landmarks())
        # Write error ellipses of landmarks.
        write_error_ellipses(f, "W E", kf.get_landmark_error_ellipses())
        # Write cylinders detected by the scanner.
        write_cylinders(f, "D C", [(obs[2][0], obs[2][1])
                                   for obs in observations])

    f.close()

### Again, a test!

In [109]:
from hashlib import shake_128
import inspect
from numpy import *
from numpy.random import default_rng
from slam_f_library import EKF_SLAM_base

def public_test(the_ekf_slam_class):
    m = shake_128()
    m.update(inspect.getsource(EKF_SLAM_base).encode())
    if m.hexdigest(8) != "05aaff349ba02a8c":
        raise ImportError("slam_f_library")
    
    # Setup filter.
    rng = default_rng()
    N = rng.integers(3,6)
    M = 3+2*N
    state = rng.uniform(0,2000,M)
    state[2] %= pi
    cov = rng.uniform(-1,1,(M, M))
    cov += cov.T + 10*eye(M)
    kf = the_ekf_slam_class(
        state.copy(),cov.copy(), 155.0, 30.0, 0.35, 0.6, 600.0, 0.7854)
    kf.number_of_landmarks = N
    li = rng.integers(N)
    kf.correct(rng.uniform(0,1,2), li)

    # Check.
    covM1 = linalg.inv(cov)
    A = covM1 - covM1@kf.covariance@covM1
    f, t = 3+2*li, 5+2*li
    landmark = state[f:t]
    H3 = EKF_SLAM_base.dh_dstate(state, landmark, 30.0)
    H2, S3 = H3[:,:2], cov[:3,:3]
    Ssl, Sll = cov[:3,f:t], cov[f:t,f:t]
    Q = diag([600.0**2, 0.7854**2])
    LM1 = linalg.inv(H3@S3@H3.T - H3@Ssl@H2.T - H2@Ssl.T@H3.T + H2@Sll@H2.T + Q)
    A[:3,:3] -= H3.T@LM1@H3
    A[:3,f:t] += H3.T@LM1@H2
    A[f:t,:3] += H2.T@LM1@H3
    A[f:t,f:t] -= H2.T@LM1@H2

    if (abs(A) > 1e-12).any():
        print("Seems to be wrong...")
        return False
    return True
assert(public_test(ExtendedKalmanFilterSLAM_3))

### And finally, let's have a look at the KF SLAM results!
You should see results identical to the ones in the video.

In [110]:
# Execute this to run the interactive viewer.
import ipy_logfile_viewer as lfv
v = lfv.IPYLogfileViewer(files=["ekf_slam_correction.txt"])

## Concluding remarks
### Understanding the results.

In [111]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/dV_5_OQbrFA" if "YouTube" in globals() else "//av.tib.eu/player/49030",
       width=560, height=315)

In [112]:
# What is better, "many" or "few" landmarks?
it_is_better_to_have = "many"
# YOUR CODE HERE
#raise NotImplementedError()

In [113]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) != reference
assert public_string_test(it_is_better_to_have.strip().lower(), '7ac66226'), "Oh no, your answer is wrong!"

### Landmark assignment and final conclusions.

In [114]:
# If you don't see a video below, run this cell.
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/UyFBDgOwZ40" if "YouTube" in globals() else "//av.tib.eu/player/49031",
       width=560, height=315)

### There's one more thing...

...one more question! Which of these alternatives is true: SLAM "must" be implemented using a Kalman filter, or SLAM "may" be implemented using a Kalman filter?

In [115]:
# SLAM "must" or "may" be implemented using a Kalman filter?
i_think_it = "may"
# YOUR CODE HERE
#raise NotImplementedError()

In [116]:
from hashlib import shake_128
def public_string_test(the_answer_string, reference):
    m = shake_128()
    m.update(the_answer_string.encode())
    return m.hexdigest(4) == reference
assert public_string_test(i_think_it.strip().lower(), '9a2119e5'), "Oh no, your answer is wrong!"